<a href="https://colab.research.google.com/github/dkurbatovv/Python/blob/main/Sport_Celebrity_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
!mkdir -p ~/.kaggle/

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d yaswanthgali/sport-celebrity-image-classification

In [ ]:
! unzip sport-celebrity-image-classification.zip

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import glob as gb

import os
import cv2
from sklearn import preprocessing
from pathlib import Path

In [ ]:
def load_data():
    dataset="/content/Sports-celebrity-images/"
    label_names = ['Kane Williamson', 'Kobe Bryant', 'Maria Sharapova', 'Ronaldo']

    images=[]
    labels=[]
    for folder in os.listdir(dataset):
        files=gb.glob(pathname=str(dataset+folder+"/*.jpg"))
        
        label = label_names.index(folder)
        for file in files:
            #read the image
            image = cv2.imread(file)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # resize the image
            image = cv2.resize(image, (256,256))

            # Append the image and its corresponding label to the output
            images.append(image)
            labels.append(label)

    images = np.array(images)
    labels = np.array(labels)
    

    return images,labels



In [ ]:
images, labels = load_data()

In [ ]:
list(images)

In [ ]:
unique, counts = np.unique(labels, return_counts=True)
print(unique)
print(counts)

In [ ]:
import seaborn as sns
label_names = ['Kane', 'Kobe', 'Maria', 'Ronaldo']
unique, counts = np.unique(labels, return_counts = True)
sns.barplot(x = label_names, y = counts)
plt.title('No of Data avaliable for each class')

In [ ]:
labels = np.array(pd.get_dummies(labels))

In [ ]:
labels 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_Test, y_train, y_Test = train_test_split( images, labels, test_size = 0.33, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_Test, y_Test, test_size = 0.5, random_state = 42)

In [ ]:
print("Length of X_train: ", len(X_train))
plt.imshow(X_train[100])
plt.title("Checking X_train"+str(y_train[100]))
plt.show()

In [ ]:
#X_train = X_train / 255
#X_Test = X_Test / 255
#X_val = X_val / 255

In [ ]:
import tensorflow

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [ ]:
from keras import layers
model = Sequential()


model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = 'same', activation = 'relu', input_shape = (256, 256, 3)))
model.add(Rescaling(1./255))
model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2,  noise_shape=None))

model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.6))
model.add(Dense(4, activation = 'softmax'))

In [ ]:
optimizer = Adam(lr = 0.001)

In [ ]:
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs = 40, validation_split = 0.2)

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(0,40)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc_train = history.history['accuracy']
acc_val = history.history['val_accuracy']
epochs = range(0,40)
plt.plot(epochs, acc_train, 'g', label='Training Accuracy')
plt.plot(epochs, acc_val, 'b', label='validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()